In [139]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [141]:
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import time
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from tensorflow.keras import backend as k

In [143]:
inicio = time.time()
inicio

1736184864.520939

In [145]:
base = pd.read_csv("autos.csv", encoding="latin1")


In [146]:
#dropando as informacoes que nao sao necessarias 
base = base.drop("dateCrawled", axis=1)
base = base.drop("dateCreated", axis=1)
base = base.drop("nrOfPictures", axis=1)
base = base.drop("postalCode", axis=1)
base = base.drop("lastSeen", axis=1)

In [147]:
base = base.drop("name", axis=1)
base = base.drop("offerType", axis=1)
base = base.drop("seller", axis=1)

In [149]:
base = base[base.price>10]
base.loc[base.price<=350000] #preco do veiculo menor ou gial a 350000 euros(nao faz sentido, tem valores invalidos)

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


In [153]:
valores = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'notRepairedDamage': 'nein'
}

In [171]:
    X = base.iloc[:,1:12].values
    y = base.iloc[:,0].values


In [175]:
X,y

(array([['test', nan, 1993, ..., 'benzin', 'volkswagen', nan],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', nan],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', nan],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990]))

In [177]:
onehotencoder = ColumnTransformer(
    transformers=[
        ("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])  # Índices das colunas para OneHotEncoder
    ],
    remainder="passthrough"  # Mantém as colunas restantes inalteradas
)

In [182]:
X = onehotencoder.fit_transform(x)  
X

array([[1.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 1.50e+05, 0.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.90e+02, 1.25e+05, 5.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.63e+02, 1.25e+05, 8.00e+00],
       ...,
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.02e+02, 1.50e+05, 3.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 1.00e+02, 1.50e+05, 6.00e+00],
       [0.00e+00, 1.00e+00, 1.00e+00, ..., 3.20e+02, 5.00e+04, 8.00e+00]])

In [185]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
    tf.keras.layers.InputLayer(shape = (316,)),
    tf.keras.layers.Dense(units = 158,activation = "relu"),#camada oculta
    tf.keras.layers.Dense(units = 158,activation = "relu"),#camada oculta
    tf.keras.layers.Dense(units = 1,activation = "linear")#camada de saida
    ])
    regressor.compile(loss="mean_absolute_error", optimizer = "Adam",metrics = ["mean_absolute_error"])
    return regressor

In [186]:
regressor = KerasRegressor(model = criar_rede,epochs=100,batch_size = 300)

In [187]:
from sklearn.linear_model import LinearRegression

# Defina o regressor
regressor = LinearRegression()


In [188]:
resultados = cross_val_score(estimator=regressor, X=X, y=y, cv=5, scoring="neg_mean_absolute_error")

In [189]:
fim = time.time()

In [191]:
(fim-inicio)/3600

0.06322017663054996

In [193]:
abs(resultados.mean())

53687.68535638042

In [195]:
abs(resultados)

array([53044.50585516, 49929.17675604, 56824.86000279, 56603.85499001,
       52036.02917792])

In [197]:
resultados.std()

2668.8553724639537